Run following to initialize environment:

In [1]:
!pip install opencv-contrib-python==4.4.0.44
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 55.7 MB 1.1 MB/s 
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
Mounted at /content/drive


Run following to extract features:

In [2]:
import os
import sys
import csv
import numpy as np
import cv2
import itertools
from google.colab.patches import cv2_imshow
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from skimage.feature import hog

def compute_sift_keypoints(gray):
  sift = cv2.SIFT_create()
  kp, des = sift.detectAndCompute(gray, None)
  return kp, des

def compute_orb_keypoints(gray):
  orb = cv2.ORB_create()
  kp, des = orb.detectAndCompute(gray, None)
  return kp, des

def feature_extraction_plan_1(X_train, Y_train, X_test, Y_test):

  def helper(X, Y):
    raw_features = []
    Y_retVal = []
    for i in range(len(X)):
      kp, des = compute_sift_keypoints(X[i])
      threshold = 5
      if len(kp) < threshold:
        continue
      kp_indices = np.argsort([x.size for x in kp])[-threshold:]
      des_chosen = des[kp_indices]
      raw_feature = np.concatenate(des_chosen)
      raw_features.append(raw_feature)
      Y_retVal.append(Y[i])
    return raw_features, Y_retVal

  train_raw_features, Y_train_retVal = helper(X_train, Y_train)
  test_raw_features, Y_test_retVal = helper(X_test, Y_test)
  pca = PCA(n_components=128)
  pca = pca.fit(train_raw_features)
  return pca.transform(train_raw_features), Y_train_retVal, pca.transform(test_raw_features), Y_test_retVal

def feature_extraction_plan_2(X_train, Y_train, X_test, Y_test):

  def helper(X, Y):
    raw_features = []
    Y_retVal = []
    for i in range(len(X)):
      kp, des = compute_sift_keypoints(X[i])
      threshold = 5
      if len(kp) < threshold:
        continue
      kp_indices = np.argpartition([x.size for x in kp], -threshold)[-threshold:]
      des_chosen = des[kp_indices]
      pca = PCA(n_components=1)
      flattened = list(itertools.chain.from_iterable(X[i]))
      raw_features.append(list(pca.fit(des_chosen).components_[0]) + flattened)
      Y_retVal.append(Y[i])
    return raw_features, Y_retVal

  train_raw_features, Y_train_retVal = helper(X_train, Y_train)
  test_raw_features, Y_test_retVal = helper(X_test, Y_test)
  #pca = PCA(n_components=128)
  #pca = pca.fit(train_raw_features)
  #return pca.transform(train_raw_features), Y_train_retVal, pca.transform(test_raw_features), Y_test_retVal
  return train_raw_features, Y_train_retVal, test_raw_features, Y_test_retVal

def feature_extraction_plan_3(X_train, Y_train, X_test, Y_test):

  feature_dimension = 128

  def helper(kmeans_obj, X, Y):
    X_retVal = []
    Y_retVal = []
    for i in range(len(X)):
      kp, des = compute_sift_keypoints(X[i])
      if len(kp) < 5:
        continue
      des_classified = kmeans_obj.predict(des)
      feature_raw = [0.0 for i in range(feature_dimension)]
      for i in range(len(des_classified)):
        feature_raw[des_classified[i]] += kp[i].response
      X_retVal.append([x / np.sum(feature_raw) for x in feature_raw])
      Y_retVal.append(Y[i])
    return X_retVal, Y_retVal

  keypoints_pool = []
  for gray in X_train:
    kp, des = compute_sift_keypoints(gray)

    # with threshold:
    #threshold = 5
    #if len(kp) < threshold:
    #  continue
    #des_chosen = des[np.argpartition([x.size for x in kp], -threshold)[-threshold:]]
    #keypoints_pool += list(des_chosen)

    # without threshold:
    if not des is None:
      keypoints_pool += list(des)

  keypoints_pool = np.array(keypoints_pool, dtype=np.float32)
  kmeans_obj = KMeans(n_clusters=feature_dimension, algorithm='full').fit(keypoints_pool)
  print(kmeans_obj.n_iter_)

  X_train_retVal, Y_train_retVal = helper(kmeans_obj, X_train, Y_train)
  X_test_retVal, Y_test_retVal = helper(kmeans_obj, X_test, Y_test)
  return X_train_retVal, Y_train_retVal, X_test_retVal, Y_test_retVal

def feature_extraction_orb(X_train, Y_train, X_test, Y_test):

  def helper(X, Y):
    raw_features = []
    Y_retVal = []
    for i in range(len(X)):
      kp, des = compute_orb_keypoints(X[i])
      threshold = 5
      if len(kp) < threshold:
        continue
      kp_indices = np.argsort([x.size for x in kp])[-threshold:]
      des_chosen = des[kp_indices]
      raw_feature = np.concatenate(des_chosen)
      raw_features.append(raw_feature)
      Y_retVal.append(Y[i])
    return raw_features, Y_retVal

  train_raw_features, Y_train_retVal = helper(X_train, Y_train)
  test_raw_features, Y_test_retVal = helper(X_test, Y_test)
  pca = PCA(n_components=128)
  pca = pca.fit(train_raw_features)
  return pca.transform(train_raw_features), Y_train_retVal, pca.transform(test_raw_features), Y_test_retVal

def feature_extraction_hog(X_train, Y_train, X_test, Y_test):
  
  def helper(X):
    retVal = []
    for x in X:
      fd = hog(x, pixels_per_cell=(2,2), cells_per_block=(1, 1), visualize=False, feature_vector=True)
      retVal.append(fd)
    return retVal

  train_fds = helper(X_train)
  test_fds = helper(X_test)
  pca = PCA(n_components=50)
  pca = pca.fit(train_fds)
  train_features = pca.transform(train_fds)
  test_features = pca.transform(test_fds)
  return train_features, Y_train, test_features, Y_test

def gen_hog_pics(X):
  samples = 5
  for i in range(samples):
    x = X[i]
    cv2.imwrite(wdir + f'/sample{i}.png', x)
    _, img = hog(x, pixels_per_cell=(2,2), cells_per_block=(1, 1), visualize=True, feature_vector=True)
    cv2.imwrite(wdir + f'/hog{i}.png', img)

def write_to_csv(X, Y, fname):
  zipped = []
  for i in range(len(X)):
    list_x = list(X[i])
    list_x.append(float(Y[i]))
    zipped.append(list_x)
  np.savetxt(os.path.join(wdir, fname), zipped, delimiter=',')

def gen_sift_pics(X):
  samples = 5
  for i in range(samples):
    x = X[i]
    kp, _ = compute_sift_keypoints(x)
    x = cv2.drawKeypoints(x, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite(wdir + f'/sift{i}.png', x)


wdir = '/content/drive/MyDrive/MLSP_Project/Code'

train_dir = os.path.join(wdir, 'sign_mnist_train')
train_file_path = os.path.join(train_dir, 'sign_mnist_train.csv')
X_train = []
Y_train = []
width = 28
height = 28

with open(train_file_path) as csvfile:
  readCSV = csv.reader(csvfile, delimiter=',')
  next(readCSV)
  for row_str in readCSV:
    row = [int(i) for i in row_str]
    Y_train.append(row[0])
    X_train.append(np.reshape(row[1:], (width, height)).astype(np.uint8))
  csvfile.close()

test_dir = os.path.join(wdir, 'sign_mnist_test')
test_file_path = os.path.join(test_dir, 'sign_mnist_test.csv')
X_test = []
Y_test = []

with open(test_file_path) as csvfile:
  readCSV = csv.reader(csvfile, delimiter=',')
  next(readCSV)
  for row in readCSV:
    Y_test.append(row[0])
    X_test.append(np.reshape(row[1:], (width, height)).astype(np.uint8))
  csvfile.close()

gen_sift_pics(X_train)
sys.exit()

X_train, Y_train, X_test, Y_test = feature_extraction_orb(X_train, Y_train, X_test, Y_test)

# sys.exit()

write_to_csv(X_train, Y_train, 'mnist_train_orb.csv')
write_to_csv(X_test, Y_test, 'mnist_test_orb.csv')

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import os
import csv
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

wdir = '/content/drive/MyDrive/MLSP_Project/Code'
#train_file_path = os.path.join(wdir, 'mnist_train_plan1.csv')
#train_file_path = os.path.join(wdir, 'mnist_train_plan2_noPCA_augmented.csv')
train_file_path = os.path.join(wdir, 'mnist_train_plan2.csv')
#train_file_path = os.path.join(wdir, 'sign_mnist_train/sign_mnist_train.csv')

#test_file_path = os.path.join(wdir, 'mnist_test_plan1.csv')
#test_file_path = os.path.join(wdir, 'mnist_test_plan2_noPCA_augmented.csv')
test_file_path = os.path.join(wdir, 'mnist_test_plan2.csv')
#test_file_path = os.path.join(wdir, 'sign_mnist_test/sign_mnist_test.csv')

X_train = []
Y_train = []
with open(train_file_path) as csvfile:
  readCSV = csv.reader(csvfile, delimiter=',')
  #next(readCSV)
  for row_str in readCSV:
    row = [float(x) for x in row_str]
    X_train.append(row[:-1])
    Y_train.append(int(row[-1]))
    #X_train.append(row[1:])
    #Y_train.append(row[0])
  csvfile.close()

X_test = []
Y_test = []
with open(test_file_path) as csvfile:
  readCSV = csv.reader(csvfile, delimiter=',')
  #next(readCSV)
  for row_str in readCSV:
    row = [float(x) for x in row_str]
    X_test.append(row[:-1])
    Y_test.append(int(row[-1]))
    #X_test.append(row[1:])
    #Y_test.append(int(row[0]))
  csvfile.close()

print(len(Y_train), len(X_train[0]))
print(len(Y_test), len(X_test[0]))

knn = KNeighborsClassifier(n_neighbors=3)
knn = knn.fit(X_train, Y_train)
Y_train_pred = knn.predict(X_train)
Y_test_pred = knn.predict(X_test)

#clf = svm.SVC(kernel='linear')
#clf.fit(X_train, Y_train)
#Y_train_pred = clf.predict(X_train)
#Y_test_pred = clf.predict(X_test)

error_num = 0
for i in range(len(Y_train)):
  if Y_train_pred[i] != Y_train[i]:
    error_num += 1
print('train error:', error_num / len(Y_train))

error_num = 0
for i in range(len(Y_test)):
  if Y_test_pred[i] != Y_test[i]:
    error_num += 1
print('test error:', error_num / len(Y_test))

21895 50
5546 50
train error: 0.018954099109385704
test error: 0.3139199423007573
